In [109]:
import pandas as pd
import time
from pymongo import MongoClient
from datetime import datetime, timedelta
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from collections import OrderedDict

In [110]:
# MongoDB 접속 정보
user = "mango"
password = "sweetmango123"
host = "j10e204.p.ssafy.io"  # 또는 MongoDB가 실행되고 있는 서버의 주소
port = 27017
dbname = "mango"  # 접속하고자 하는 데이터베이스 이름

# MongoDB URI 구성
uri = f"mongodb://{user}:{password}@{host}:{port}"

# MongoDB 클라이언트 생성
client = MongoClient(uri)

# 데이터베이스 선택
db = client[dbname]
# 접속 테스트를 위한 컬렉션 및 데이터 출력 예시
collection = db["news"]  # 사용할 컬렉션 이름

In [123]:
# 오늘 날짜와 정확히 1년 전 날짜 계산
today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
one_year_ago = today - timedelta(days=366)

# 날짜를 "YYYY-MM-DD" 형식의 문자열로 변환
one_year_ago_str = one_year_ago.strftime("%Y-%m-%d")

# 쿼리 실행 전 시간 측정
start_time = time.time()

## published 필드 기준으로 인덱싱
# collection.create_index([("published", 1)])
# 문자열 기반으로 1년 전 날짜에 해당하는 기사 조회
last_year_news = list(collection.find({"published": {"$regex": f"^{one_year_ago_str}"}}, {"title": 1, "_id": 0}))
# 뉴스 데이터의 'article' 필드만 추출하여 리스트 생성
titles = [news['title'] for news in last_year_news if 'title' in news]
print(len(titles))
# 쿼리 실행 후 시간 측정
end_time = time.time()
# 실행 시간 출력
print(f"쿼리 실행 시간: {end_time - start_time}초")

7144
쿼리 실행 시간: 0.3420555591583252초


In [113]:
unique_titles = list(OrderedDict.fromkeys(titles))

In [114]:
len(unique_titles)

6006

## 불용어 불러오기

In [116]:
file_path = 'korean_stopwords.txt'  # 여기에 파일 경로를 입력하세요
with open(file_path, 'r', encoding='utf-8') as file:
    # 파일 내용 읽기
    content = file.read()
stopwords = content.strip().split('\n')
stopwords

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [117]:
# 뉴스 본문에서 불용어 제거
cleaned_titles = []
for title in unique_titles:
    words = title.split()  # 공백 기준으로 단어 분리
    filtered_words = [word for word in words if word not in stopwords]  # 불용어가 아닌 단어만 선택
    cleaned_title = ' '.join(filtered_words)  # 정제된 단어들을 다시 문자열로 결합
    cleaned_titles.append(cleaned_title)

In [118]:
cleaned_titles[0]

'국보법 위반, 5범 후보…‘전과‘에 몸살 앓는 4·5 재보선'

In [119]:
tfidf_vectorizer = TfidfVectorizer()

# TF-IDF 계산
tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_titles)

# 각 단어의 TF-IDF 점수를 기준으로 정렬하여 상위 N개 키워드 추출
def extract_top_n_keywords(tfidf_matrix, feature_names, n=10):
    # 각 기사별로 반복
    for i in range(tfidf_matrix.shape[0]):
        # 해당 기사의 모든 단어의 TF-IDF 점수 추출
        scores = tfidf_matrix[i].toarray().flatten()
        # 점수에 따라 단어 인덱스 정렬
        sorted_indices = np.argsort(scores)[::-1]
        top_n_indices = sorted_indices[:n]
        # 상위 N개 키워드와 그 점수 출력
        print(f"Article {i+1} Top-{n} keywords:")
        for index in top_n_indices:
            print(f"{feature_names[index]} ({scores[index]:.4f})")
        print("\n")

# TF-IDF 벡터라이저에서 단어 목록 추출
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

# 상위 N개 키워드 추출 함수 호출
extract_top_n_keywords(tfidf_matrix, feature_names, n=10)

Article 1 Top-10 keywords:
5범 (0.3801)
재보선 (0.3801)
앓는 (0.3801)
국보법 (0.3801)
전과 (0.3630)
몸살 (0.3630)
위반 (0.2980)
후보 (0.2644)
들려온다 (0.0000)
뒷말 (0.0000)


Article 2 Top-10 keywords:
원한다 (0.3787)
키워 (0.3617)
세상보다 (0.3617)
키울 (0.3496)
세상 (0.3402)
주는 (0.3325)
내가 (0.3070)
사설 (0.2913)
있는 (0.2596)
두자릿수 (0.0000)


Article 3 Top-10 keywords:
배터리 (0.5172)
굴기 (0.3474)
美에 (0.3474)
맞설 (0.3317)
기지로 (0.3317)
lg엔솔 (0.2939)
사설 (0.2672)
공장 (0.2606)
최대 (0.2131)
둘러싼 (0.0000)


Article 4 Top-10 keywords:
받겠다는 (0.4236)
mz노조 (0.4045)
보조금 (0.4045)
맞다 (0.4045)
이게 (0.3805)
사설 (0.3258)
정부 (0.2804)
李대표 (0.0000)
뒤쪽의 (0.0000)
뒤처진 (0.0000)


Article 5 Top-10 keywords:
순국 (0.4432)
113주기 (0.4432)
안중근 (0.4284)
의사 (0.4075)
추모식 (0.4075)
포토 (0.3025)
뒷동산 (0.0000)
뒤로 (0.0000)
뒤에 (0.0000)
뒤에도 (0.0000)


Article 6 Top-10 keywords:
추위 (0.4202)
보통 (0.4202)
반짝 (0.4120)
미세먼지 (0.3681)
아침 (0.3649)
오늘의 (0.3403)
날씨 (0.3045)
뒷동산 (0.0000)
뒤로 (0.0000)
뒤에 (0.0000)


Article 7 Top-10 keywords:
느껴지면 (0.3988)
계절성 (0.3988)
타나 (0.3988)
주의보 (0.

In [120]:
# TfidfVectorizer 초기화
tfidf_vectorizer = TfidfVectorizer()

# TF-IDF 계산
tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_titles)

# 전체 문서에 대한 각 단어의 TF-IDF 점수 합산
scores_sum = np.sum(tfidf_matrix, axis=0)

# 배열 형태로 변환
scores_sum_array = np.array(scores_sum).flatten()

# 피처 이름 가져오기
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

# 점수에 따라 내림차순으로 정렬하고 상위 N개의 인덱스 가져오기
top_n_indices = np.argsort(scores_sum_array)[::-1][:10]

# 상위 N개 키워드와 점수 출력
print("Top N Keywords:")
for index in top_n_indices:
    print(f"{feature_names[index]} ({scores_sum_array[index]:.4f})")

Top N Keywords:
급증 (112.9889)
매도잔량 (75.6027)
오르며 (57.0777)
체결강도 (57.0777)
매수잔량 (57.0368)
지속 (42.4047)
강세 (41.3144)
2023 (33.7808)
27 (30.4585)
03 (30.0200)


In [127]:
# 형태소 분석기 초기화
okt = Okt()

# 명사만 추출하는 함수
def get_nouns(text):
    nouns = okt.nouns(text)
    return [noun for noun in nouns if len(noun) > 2]  # 2글자 명사 제외

# TfidfVectorizer 초기화 (명사만 추출하여 사용)
tfidf_vectorizer = TfidfVectorizer(tokenizer=get_nouns, min_df=10)  # min_df는 최소 문서 빈도, 필요에 따라 조정

# TF-IDF 계산
tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_titles)  # cleaned_articles는 전처리된 뉴스 본문 리스트

# 전체 문서에 대한 각 단어의 TF-IDF 점수 합산
scores_sum = np.sum(tfidf_matrix, axis=0)

# 배열 형태로 변환
scores_sum_array = np.array(scores_sum).flatten()

# 피처 이름 가져오기
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

# 점수에 따라 내림차순으로 정렬하고 상위 N개의 인덱스 가져오기
top_n_indices = np.argsort(scores_sum_array)[::-1][:3]  # 상위 10개 키워드

# 상위 N개 키워드와 점수 출력
result = []
print("Top N Keywords:")
for index in top_n_indices:
    result.append(feature_names[index])
result

c:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Top N Keywords:


['유아인', '한동훈', '이재명']